# 多クラス分類の評価指標

## 混同行列(Confusion Matrics)

混同行列とは，分類問題で出力されたクラス分類の結果をまとめたマトリックスのこと．  
2クラスの場合，実際のクラスとモデルによって予測されたクラスはそれぞれ2つであるため，2×2の行列となる

この4つのクラスを用いて，次節以降の評価指標を導出することができる

![](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/70152/29cc37e7-ff2e-d781-d4a4-e328fd7526a1.png)
参考：[https://qiita.com/TsutomuNakamura/items/a1a6a02cb9bb0dcbb37f](https://qiita.com/TsutomuNakamura/items/a1a6a02cb9bb0dcbb37f)

## 正答率(Accuracy)

データ全体の中で，予測が正解した割合  
高いほど高性能なモデルである  

$$
Accuracy = \frac{TP + TN}{TP + FP + FN + TN}
$$

## 適合率(Precision)

Positive と分類されたデータ(TP + FP)の中で実際にPositiveだったデータ(TP)数の割合  
高いほどPositiveの判定の精度がよい

$$
Precision = \frac{TP}{TP + FP}
$$

## 再現率(Recall)

本来Positiveなデータを正しくPositiveと推測できた割合  
高いほどPositiveなデータを取りこぼしなく回収できている

$$
Recall = \frac{TP}{TP + FN}
$$

## 交差エントロピー誤差(Cross Entropy Loss; LogLoss)

交差エントロピーとは，確率分布pと確率分布qの近似性を表現する関数．言い換えると，予測が真の分布からどれほど「間違っているか」を表す．次式で定義される  

$$
E = -\sum_{k}q(k)\log(p(k))\\
p：学習モデルによる確率分布　q：教師データの確率分布
$$

ここで，分類問題について考えると，qは全て0か1になるため，正解したデータの項のみが残り，教師データとの誤差として扱うことができる．
簡素化したのが次式

$$
E = -\log(p(k))
$$
![](https://manareki.com/wp-content/uploads/2019/03/log.png)

グラフとしてはこのようになり，この誤差関数を損失関数として用いて最適化を行うことで，学習モデルの正答率p(k)を向上させることができる

＜特性＞
- 教師データと学習データの乖離が激しいとき，二乗誤差関数などと比べて学習スピードが速い  
(学習の速さは損失関数の微分値に依存する)

参考サイト：[https://manareki.com/crossentropy_lossfunction](https://manareki.com/crossentropy_lossfunction)

## sklearnを用いた各評価指標の計算

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# データの読み込み
from sklearn.datasets import load_breast_cancer
bcan = load_breast_cancer()
X = bcan.data
y = bcan.target
df = pd.DataFrame(X, columns=bcan.feature_names)
df['y'] = y
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,y
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
# 数値モデルの構築
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['y']), df['y'], train_size=0.7, random_state=0)
print(X_train.shape, X_test.shape)

(398, 30) (171, 30)


In [4]:
# 予測モデルの構築
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression(C=100)
lreg.fit(X_train, y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
# モデルの評価
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, log_loss
y_pred = lreg.predict(X_test)
y_pred_list = lreg.predict_proba(X_test)

print(f'ConfusionMatrics\n{confusion_matrix(y_test, y_pred)}\n')
print(f' Accuracy: {accuracy_score(y_test, y_pred):.3f}')
print(f'Precision: {precision_score(y_test, y_pred):.3f}')
print(f'   Recall: {recall_score(y_test, y_pred):.3f}')
print(f' CEntropy: {log_loss(y_test, y_pred_list):.3f}')
lreg.score(X_test, y_test)

ConfusionMatrics
[[ 62   1]
 [  6 102]]

 Accuracy: 0.959
Precision: 0.990
   Recall: 0.944
 CEntropy: 0.099


0.9590643274853801